In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# 시드설정
random.seed(42)
torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed_all(42)

cuda


### Get data

In [3]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
# Set parameter

learning_rate = 0.001
training_epochs = 100
batch_size = 32

### Create Data Loader

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

### Create layers

In [6]:
linear1 = torch.nn.Linear(784, 128, bias=True)
linear2 = torch.nn.Linear(128, 4, bias=True)
linear3 = torch.nn.Linear(4, 10, bias=True)

sigmoid = torch.nn.Sigmoid() #activation function으로 ReLU 설정

In [7]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.2535, -0.5273,  0.0695,  0.0558],
        [ 0.4512,  0.3165, -0.3048, -0.5822],
        [ 0.0449,  0.4054, -0.0987,  0.3202],
        [ 0.0657, -0.2631, -0.1646,  0.1454],
        [ 0.5175, -0.6424, -0.6172, -0.5183],
        [ 0.0917,  0.0590, -0.1163, -0.4603],
        [ 0.1879,  0.1701, -0.0626,  0.2943],
        [-0.1619, -0.2485, -0.4934, -0.0488],
        [ 0.0555,  0.2764, -0.2809,  0.2704],
        [-0.5093, -0.0135, -0.4086, -0.3675]], requires_grad=True)

In [14]:
# Sequential model

model = torch.nn.Sequential(linear1, sigmoid,
                            linear2, sigmoid,
                            linear3
                            ).cuda()

In [15]:
#Define cost F and optimizer

error = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = error(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.069664210
Epoch: 0002 cost = 0.058785841
Epoch: 0003 cost = 0.050410934
Epoch: 0004 cost = 0.043329697
Epoch: 0005 cost = 0.037601035
Epoch: 0006 cost = 0.033379007
Epoch: 0007 cost = 0.029053776
Epoch: 0008 cost = 0.025284989
Epoch: 0009 cost = 0.022624180
Epoch: 0010 cost = 0.019337805
Learning finished


In [24]:
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    # 임의의 자료 가져와 일치여부 확인
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data.to(device))
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9488999843597412
Label:  6
Prediction:  6


/home/mglee/.conda/envs/svmglee/lib/python3.8/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/mglee/.conda/envs/svmglee/lib/python3.8/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
